2 CNN: 0.98
1 CNN: 0.9615 epoch 3
1 CNN: 0.9782 epoch 50
ABC: 0.9575 batch 100, lr 0.0001, metrics criterion 'CE', epoch 3
ABC: 0.9671 batch 100, lr 0.0001, metrics criterion 'CE', epoch 6
ABC: 0.9768 epoch 50 kernel_per_pixel 8

In [1]:
import torch
import torchvision
import torch.nn.functional as F
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from utils.tools import dotdict
from driver.driver import ABC_Driver
from torch_geometric_temporal import METRLADatasetLoader
from other_model.other_model import make_default_model
import atd2022
torch.cuda.is_available()
torch.cuda.set_device(0)

In [2]:
atd_args = dotdict()

atd_args.name = 'atd'
atd_args.train_batch_size = 25
atd_args.predict_len = 4
atd_args.history_len = 7

atd_args.train_epochs= 10
atd_args.lr = 0.00009
atd_args.criterion = 'L1'
atd_args.scheduler = None

activation = 'relu'
pool_name = 'avg'
input_channel = 1
pixel_number = atd_args.history_len*5200
kernel_size = 6
knpp = 12
knpp2 = atd_args.predict_len

atd_args.layers=[
    ('agnostic', ((input_channel, knpp, kernel_size), 1, None, None, activation)),
    ('agnostic', ((knpp, knpp2, kernel_size), 1, None, None, activation)),
    ('linear', (knpp2*atd_args.history_len, atd_args.predict_len, (1,2), 1, (1, atd_args.predict_len))),
]

In [3]:
wt_args = dotdict()

wt_args.name = 'wiki_traffic'
wt_args.train_batch_size = 30
wt_args.predict_len = 1
wt_args.history_len = 5

wt_args.train_epochs= 10
wt_args.lr = 0.001
wt_args.criterion = 'L1'
wt_args.scheduler = None

activation = 'relu'
pool_name = 'avg'
input_channel = 1
pixel_number = wt_args.history_len*1400
kernel_size = 3
knpp = 5
knpp2 = 15

wt_args.layers=[
    ('specific', ((input_channel, knpp, kernel_size), 1, None, None, activation)),
    ('specific', ((knpp, knpp2, kernel_size), 1, None, None, activation)),
    ('linear', (knpp2*wt_args.history_len, wt_args.predict_len, (1,2), 1, (1, wt_args.predict_len))),
]

In [4]:
lat_args = dotdict()

lat_args.name = 'lat'
lat_args.train_batch_size = 30
lat_args.predict_len = 1
lat_args.history_len = 5

lat_args.train_epochs= 10
lat_args.lr = 0.001
lat_args.criterion = 'L1'
lat_args.scheduler = None

activation = 'relu'
pool_name = 'avg'
input_channel = 1
pixel_number = lat_args.history_len*207
kernel_size = 3
knpp = 5
knpp2 = 15

lat_args.layers=[
    ('specific', ((input_channel, knpp, kernel_size), 1, None, None, activation)),
    ('specific', ((knpp, knpp2, kernel_size), 1, None, None, activation)),
    ('linear', (knpp2*lat_args.history_len, lat_args.predict_len, (1,2), 1, (1, lat_args.predict_len))),
]

In [5]:
mnist_args = dotdict()

mnist_args.name = 'mnist'
mnist_args.train_batch_size = 120
mnist_args.predict_batch_size = 100

mnist_args.train_epochs = 30
mnist_args.lr = 0.001
mnist_args.criterion = 'CE'
mnist_args.optimizer = 'Adam'
mnist_args.scheduler = 'multistep3'

activation = 'relu'
input_channel = 1
knpp = [30,60,120,180,240]

mnist_args.layers=[
    ('cnn2d', ((input_channel, knpp[0], (7,7), 1, 3, 1, 1), 1, None, None, activation, False)),
    ('atc2d', ((knpp[0], knpp[1], (3,3), 1, 1, 1, knpp[0]), 1, None, None, activation, True)),
    ('atc2d', ((knpp[1], knpp[2], (3,3), 1, 1, 1, knpp[0]), 1, 'first', (2,2), activation, True)),
    ('atc2d', ((knpp[2], knpp[3], (3,3), 1, 1, 1, knpp[0]), 1, None, None, activation, True)),
    ('atc2d', ((knpp[3], knpp[4], (3,3), 1, 1, 1, knpp[0]), 1, 'first', (2,2), activation, True)),
    ('cnn2d', ((knpp[-1], knpp[-1], (3,3), 1, 1, 1, knpp[0]), 1, None, None, activation, False)),
    ('cnn2d', ((knpp[-1], knpp[-1], (3,3), 1, 0, 1, knpp[0]), 1, None, None, False, False)),
    ('adptavgpool', (1,1)),
    ('linear', (knpp[-1], 10, (1,2,3))),
    ('softmax', (1))
]

# knpp = [24,48,96,192]
# # knpp = [30,60,120,240]

# mnist_args.layers=[
#     ('cnn2d', ((input_channel, knpp[0], (3,3), 1, 1, 1, 1), 1, None, None, activation, False)),
#     ('atc2d', ((knpp[0], knpp[1], (3,3), 1, 1, 1, knpp[0]), 1, None, None, activation, True)),
#     ('atc2d', ((knpp[1], knpp[2], (3,3), 1, 1, 1, knpp[0]), 1, 'first', (2,2), activation, True)),
#     ('atc2d', ((knpp[2], knpp[3], (3,3), 1, 1, 1, knpp[0]), 1, 'first', (2,2), activation, True)),
#     ('cnn2d', ((knpp[-1], knpp[-1], (3,3), 1, 0, 1, knpp[0]), 1, None, None, activation, False)),
#     ('cnn2d', ((knpp[-1], knpp[-1], (3,3), 1, 0, 1, knpp[0]), 1, None, None, False, False)),
#     ('adptavgpool', (1,1)),
#     ('linear', (knpp[-1], 10, (1,2,3))),
#     ('softmax', (1))
# ]

In [12]:
cifar10_args = dotdict()

cifar10_args.name = 'cifar10'
cifar10_args.train_batch_size = 128
cifar10_args.predict_batch_size = 100

cifar10_args.train_epochs = 250
cifar10_args.lr = 0.001
cifar10_args.criterion = 'CE'
cifar10_args.optimizer = 'Adam'
cifar10_args.scheduler = 'multistep'

activation = 'relu'
input_channel = 3
knpp = [40,80,160,240,320]

# cifar10_args.layers=[
#     ('cnn2d', ((input_channel, knpp[0], (3,3), 1, 1, 1, 1), 1, None, None, activation, False)),
#     ('atc2d', ((knpp[0], knpp[1], (3,3), 1, 1, 1, knpp[0]), 1, None, None, activation, True)),
#     ('atc2d', ((knpp[1], knpp[2], (3,3), 1, 1, 1, knpp[0]), 1, 'first', (2,2), activation, True)),
#     ('atc2d', ((knpp[2], knpp[3], (3,3), 1, 1, 1, knpp[0]), 1, None, None, activation, True)),
#     ('atc2d', ((knpp[3], knpp[4], (3,3), 1, 1, 1, knpp[0]), 1, 'first', (2,2), activation, True)),
#     ('cnn2d', ((knpp[-1], knpp[-1], (3,3), 1, 1, 1, knpp[0]), 1, None, None, activation, False)),
#     ('cnn2d', ((knpp[-1], knpp[-1], (3,3), 1, 0, 1, knpp[0]), 1, None, None, False, False)),
#     ('adptavgpool', (1,1)),
#     ('linear', (knpp[-1], 10, (1,2,3))),
#     ('softmax', (1))
# ]

cifar10_args.layers=[('cnn2d', ((3, 16, (3, 3), 1, 1), 1, None, None, 'relu', False)), 
 ('cnn2d', ((16, 16, (3, 3), 1, 1), 2, None, None, 'relu')), 
 ('cnn2d', ((16, 16, (3, 3), 1, 1), 2, None, None, 'relu')), 
 ('cnn2d', ((16, 16, (3, 3), 1, 1), 2, None, None, 'relu')), 
 ('cnn2d', ((16, 32, (3, 3), 1, 1), 2, 'first', (2, 2), 'relu')), 
 ('cnn2d', ((32, 32, (3, 3), 1, 1), 2, None, None, 'relu')), 
 ('cnn2d', ((32, 32, (3, 3), 1, 1), 2, None, None, 'relu')), 
 ('cnn2d', ((32, 64, (3, 3), 1, 1), 2, 'first', (2, 2), 'relu')), 
 ('cnn2d', ((64, 64, (3, 3), 1, 1), 2, None, None, 'relu')), 
 ('cnn2d', ((64, 64, (3, 3), 1, 1), 2, None, None, 'relu')), 
 ('adptavgpool', (1, 1)), 
 ('linear', (64, 10, (1, 2, 3))), 
 ('softmax', 1)]

In [13]:
args = cifar10_args
data = None
# data = atd2022.io.read_csv()
# data = pd.read_csv('/scratch/mfeng/data/ABC/Wiki_Traffic/filled_selected_train_1.csv', index_col=0, header=[0,1,2,3], parse_dates=True)
# data = pd.read_csv('/scratch/mfeng/data/ABC/LA_Traffic/LA_Traffic.csv', index_col=0)
# data = data.head(200)

driver = ABC_Driver(args, data, record_path=None, if_hash=False)
# driver.train()

Use GPU
Files already downloaded and verified
Files already downloaded and verified
add record: 02/20/2023 23:59


In [8]:
driver.train()

epoch: 0, train_loss: 1.6181, test_metric: 0.8198, time: 3627.809912443161
epoch: 1, train_loss: 1.6144, test_metric: 0.8241, time: 318.12082505226135
epoch: 2, train_loss: 1.6136, test_metric: 0.8213, time: 317.9991126060486
epoch: 3, train_loss: 1.6097, test_metric: 0.84, time: 318.2684280872345
epoch: 5, train_loss: 1.6043, test_metric: 0.8353, time: 318.33043360710144
epoch: 6, train_loss: 1.6029, test_metric: 0.8401, time: 318.1367197036743
epoch: 7, train_loss: 1.5976, test_metric: 0.8346, time: 318.0952470302582
epoch: 8, train_loss: 1.5967, test_metric: 0.8365, time: 318.26040410995483
epoch: 9, train_loss: 1.5933, test_metric: 0.8494, time: 317.99820017814636
epoch: 10, train_loss: 1.5913, test_metric: 0.8407, time: 318.1279516220093
epoch: 11, train_loss: 1.5877, test_metric: 0.8411, time: 318.28657937049866
epoch: 12, train_loss: 1.5894, test_metric: 0.8469, time: 318.07630729675293
epoch: 13, train_loss: 1.5864, test_metric: 0.8458, time: 317.9950020313263
epoch: 14, train_

epoch: 109, train_loss: 1.4773, test_metric: 0.8959, time: 317.66199350357056
epoch: 110, train_loss: 1.4775, test_metric: 0.8982, time: 317.50482988357544
epoch: 111, train_loss: 1.4766, test_metric: 0.8958, time: 318.06205224990845
epoch: 112, train_loss: 1.4766, test_metric: 0.8983, time: 317.71575808525085
epoch: 113, train_loss: 1.4773, test_metric: 0.8957, time: 318.68185448646545
epoch: 114, train_loss: 1.4757, test_metric: 0.8984, time: 317.9829523563385
epoch: 115, train_loss: 1.476, test_metric: 0.8983, time: 317.66626381874084
epoch: 116, train_loss: 1.4763, test_metric: 0.898, time: 318.401243686676
epoch: 117, train_loss: 1.4766, test_metric: 0.8989, time: 318.24490189552307
epoch: 118, train_loss: 1.4768, test_metric: 0.9005, time: 318.0798223018646
epoch: 119, train_loss: 1.476, test_metric: 0.9035, time: 318.16820096969604
epoch: 120, train_loss: 1.474, test_metric: 0.901, time: 318.72754669189453
epoch: 121, train_loss: 1.4727, test_metric: 0.9016, time: 318.1766674518

epoch: 216, train_loss: 1.4679, test_metric: 0.9023, time: 318.9922752380371
epoch: 217, train_loss: 1.4677, test_metric: 0.9029, time: 319.5867464542389
epoch: 218, train_loss: 1.4676, test_metric: 0.9025, time: 319.05059146881104
epoch: 219, train_loss: 1.4679, test_metric: 0.9029, time: 319.1204822063446
epoch: 220, train_loss: 1.4676, test_metric: 0.9039, time: 320.11825609207153
epoch: 221, train_loss: 1.4681, test_metric: 0.9036, time: 319.61707639694214
epoch: 222, train_loss: 1.4676, test_metric: 0.9036, time: 319.55531191825867
epoch: 223, train_loss: 1.4675, test_metric: 0.9023, time: 319.6396834850311
epoch: 224, train_loss: 1.4676, test_metric: 0.9045, time: 319.4268012046814
epoch: 225, train_loss: 1.4678, test_metric: 0.902, time: 319.64025807380676
epoch: 226, train_loss: 1.4676, test_metric: 0.9032, time: 319.2918527126312
epoch: 227, train_loss: 1.4676, test_metric: 0.9028, time: 318.93611574172974
epoch: 228, train_loss: 1.4678, test_metric: 0.9037, time: 319.63782429

In [14]:
pytorch_total_params = sum(p.numel() for p in driver.model.parameters() if p.requires_grad)

In [15]:
pytorch_total_params

272570

In [ ]:

# cifar10_args.layers=[
#     ('agnostic', ((input_channel, knpp[0], 9), 1, None, None, activation, False)),
#     ('agnostic', ((knpp[0], knpp[0], 9), 2, None, None, activation)),
#     ('agnostic', ((knpp[0], knpp[1], 9), 2, None, None, activation)),
#     ('agnostic', ((knpp[1], knpp[2], 9), 2, None, None, activation)),
#     ('agnostic', ((knpp[2], knpp[3], 9), 2, 'first', (2,2), activation)),
#     ('agnostic', ((knpp[3], knpp[4], 9), 2, None, None, activation)),
#     ('agnostic', ((knpp[4], knpp[5], 9), 2, None, None, activation)),
#     ('agnostic', ((knpp[5], knpp[6], 9), 2, 'first', (2,2), activation)),
#     ('agnostic', ((knpp[6], knpp[7], 9), 2, None, None, activation)),
#     ('agnostic', ((knpp[7], knpp[8], 9), 2, None, None, activation)),
#     ('adptavgpool', (1,1)),
#     ('linear', (knpp[-1], 10, (1,2,3))),
#     ('softmax', (1))
# ]

# cifar10_args.layers=[
#     ('cnn2d', ((input_channel, knpp[0], (3,3), 1, 1), 1, None, None, activation, False)),
#     ('cnn2d', ((knpp[0], knpp[0], (3,3), 1, 1), 2, None, None, activation)),
#     ('cnn2d', ((knpp[0], knpp[1], (3,3), 1, 1), 2, None, None, activation)),
#     ('cnn2d', ((knpp[1], knpp[2], (3,3), 1, 1), 2, None, None, activation)),
#     ('cnn2d', ((knpp[2], knpp[3], (3,3), 1, 1), 2, 'first', (2,2), activation)),
#     ('cnn2d', ((knpp[3], knpp[4], (3,3), 1, 1), 2, None, None, activation)),
#     ('cnn2d', ((knpp[4], knpp[5], (3,3), 1, 1), 2, None, None, activation)),
#     ('cnn2d', ((knpp[5], knpp[6], (3,3), 1, 1), 2, 'first', (2,2), activation)),
#     ('cnn2d', ((knpp[6], knpp[7], (3,3), 1, 1), 2, None, None, activation)),
#     ('cnn2d', ((knpp[7], knpp[8], (3,3), 1, 1), 2, None, None, activation)),
#     ('adptavgpool', (1,1)),
#     ('linear', (knpp[-1], 10, (1,2,3))),
#     ('softmax', (1))
# ]

In [ ]:
[('cnn2d', ((3, 16, (3, 3), 1, 1), 1, None, None, 'relu', False)), 
 ('cnn2d', ((16, 16, (3, 3), 1, 1), 2, None, None, 'relu')), 
 ('cnn2d', ((16, 16, (3, 3), 1, 1), 2, None, None, 'relu')), 
 ('cnn2d', ((16, 16, (3, 3), 1, 1), 2, None, None, 'relu')), 
 ('cnn2d', ((16, 32, (3, 3), 1, 1), 2, 'first', (2, 2), 'relu')), 
 ('cnn2d', ((32, 32, (3, 3), 1, 1), 2, None, None, 'relu')), 
 ('cnn2d', ((32, 32, (3, 3), 1, 1), 2, None, None, 'relu')), 
 ('cnn2d', ((32, 64, (3, 3), 1, 1), 2, 'first', (2, 2), 'relu')), 
 ('cnn2d', ((64, 64, (3, 3), 1, 1), 2, None, None, 'relu')), 
 ('cnn2d', ((64, 64, (3, 3), 1, 1), 2, None, None, 'relu')), 
 ('adptavgpool', (1, 1)), 
 ('linear', (64, 10, (1, 2, 3))), 
 ('softmax', 1)]